<a href="https://colab.research.google.com/github/czovekboti/chess_rl/blob/sft%2Bgrpo/Final_models_eval" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
  https://colab.research.google.com/github/czovekboti/chess_rl/blob/sft%2Bgrpo/Model%20Evaluation.ipynb
)


In [ ]:
#@title Colab Extra Install { display-mode: "form" }
#%%capture
!pip install unsloth python-chess datasets matplotlib pandas tqdm python-dotenv colorama transformers
!pip install python-chess
!apt-get install stockfish

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 75.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.7/359.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━

# Add how many examples the script should eval

In [ ]:
import torch
import chess
import chess.engine
import re
import pandas as pd
from datasets import load_dataset
from unsloth import FastLanguageModel
from tqdm import tqdm
import json
import time

SYSTEM_PROMPT = """
You are a chess coach assistant. You will be given a board position in FEN format. Your job is to analyze the board and suggest the best legal move for the player whose turn it is.

Please follow this exact format in your response:

<reasoning>
(Brief explanation of what you see on the board — piece activity, threats, and candidate moves)
</reasoning>
<answer>
(best move written in correct SAN format, such as Nf3 or exd5)
</answer>

Do not invent illegal or impossible moves. The move must be legal in the given FEN position.
Do not use UCI format like e2e4 — only SAN notation like e4, Nf3, or O-O.
In case of taking a piece use the [file]x[target square] format
### Example:
FEN: rnbqkbnr/pppppppp/8/8/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 1

<reasoning>
White has just played e4 and developed the knight to f3. It's Black's turn. The e4 pawn is undefended. Capturing it with the pawn from d7 to d5 is a natural central counter.
</reasoning>
<answer>
d5
</answer>

Now solve the following position:
"""

class ChessModelEvaluator:
    def __init__(self, model_path, model_name, stockfish_path="/usr/games/stockfish",
                 max_seq_length=1024, lora_rank=32, num_generations=1,
                 model=None, tokenizer=None):
        """
        Initialize the evaluator with a model.

        Args:
            model_path: Path to the model (ignored if model and tokenizer provided)
            model_name: Name for saving results (e.g., "sft_model", "grpo_model")
            stockfish_path: Path to stockfish executable
            max_seq_length: Maximum sequence length for model
            lora_rank: LoRA rank for model loading
            num_generations: Number of generations per position
            model: Pre-loaded model (optional, avoids reloading)
            tokenizer: Pre-loaded tokenizer (optional, avoids reloading)
        """
        self.model_name = model_name
        self.num_generations = num_generations

        # Load model if not provided
        if model is not None and tokenizer is not None:
            print(f"Using pre-loaded model for: {model_name}")
            self.model = model
            self.tokenizer = tokenizer
        else:
            print(f"Loading model: {model_path}")
            self.model, self.tokenizer = FastLanguageModel.from_pretrained(
                model_name=model_path,
                max_seq_length=max_seq_length,
                load_in_4bit=False,
                fast_inference=False,
                max_lora_rank=lora_rank,
                gpu_memory_utilization=0.8,
                resize_model_vocab=151669,
            )
            FastLanguageModel.for_inference(self.model)

        # Initialize chess engine
        print(f"Initializing Stockfish from: {stockfish_path}")
        self.engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

    def extract_move_from_response(self, response):
        """Extract move from model response."""
        match = re.search(r'<answer>\s*([^\s<]+)\s*</answer>', response, re.IGNORECASE)
        if match:
            return match.group(1).strip()
        return None

    def has_correct_format(self, response):
        """Check if response has correct format with reasoning and answer tags."""
        has_reasoning = bool(re.search(r'<reasoning>.*?</reasoning>', response, re.IGNORECASE | re.DOTALL))
        has_answer = bool(re.search(r'<answer>.*?</answer>', response, re.IGNORECASE | re.DOTALL))
        return has_reasoning and has_answer

    def fen_color(self, fen):
        """Extract which color is to move from FEN."""
        parts = fen.split()
        if len(parts) >= 2:
            return "White" if parts[1] == 'w' else "Black"
        return "White"  # Default

    def get_game_phase(self, board):
        """Determine game phase based on material and move count."""
        move_count = board.fullmove_number
        piece_count = len(board.piece_map())

        if move_count <= 10 or piece_count >= 28:
            return "opening"
        elif piece_count <= 12 or move_count >= 40:
            return "endgame"
        else:
            return "middlegame"

    def count_material(self, board):
        """Count total material on board."""
        piece_values = {'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 0}
        total = 0
        for piece in board.piece_map().values():
            total += piece_values.get(piece.symbol().upper(), 0)
        return total

    def is_tactical(self, board):
        """Check if position is tactical (checks, captures available)."""
        if board.is_check():
            return True
        # Check if there are any capture moves
        for move in board.legal_moves:
            if board.is_capture(move):
                return True
        return False

    def get_position_type(self, best_eval, top_evals):
        """Classify position type based on evaluation."""
        if abs(best_eval) > 300:
            return "winning" if best_eval > 0 else "losing"
        elif abs(best_eval) < 50:
            return "equal"
        else:
            return "advantage"

    def classify_move_quality(self, cp_loss):
        """Classify move quality based on centipawn loss."""
        if cp_loss is None:
            return "illegal"
        elif cp_loss <= 10:
            return "excellent"
        elif cp_loss <= 50:
            return "good"
        elif cp_loss <= 100:
            return "inaccuracy"
        elif cp_loss <= 300:
            return "mistake"
        else:
            return "blunder"

    def generate_move(self, fen):
        """Generate a move for the given FEN position."""
        # Format prompt exactly as during training
        messages = [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': fen + " You are with the following pieces: " + self.fen_color(fen)}
        ]

        # Apply chat template
        prompt = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        inputs = self.tokenizer([prompt], return_tensors="pt").to("cuda")

        start_time = time.time()
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
        )
        inference_time = time.time() - start_time

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the generated part (after the prompt)
        response = response[len(prompt):].strip()

        # Count tokens in response
        response_tokens = len(self.tokenizer.encode(response))

        return response, inference_time, response_tokens

    def evaluate_position(self, fen, ground_truth_eval=None):
        """
        Evaluate model's move for a given position.

        Returns dict with evaluation metrics for this position.
        """
        board = chess.Board(fen)
        phase = self.get_game_phase(board)
        material = self.count_material(board)
        is_tactical = self.is_tactical(board)
        move_number = board.fullmove_number

        # Get top moves from Stockfish
        info = self.engine.analyse(board, chess.engine.Limit(depth=20), multipv=5)

        # Extract top moves and their evaluations
        top_moves = []
        top_evals = []
        for i, analysis in enumerate(info if isinstance(info, list) else [info]):
            move = analysis["pv"][0]
            score = analysis["score"].relative
            top_moves.append(board.san(move))
            # Convert score to centipawns
            if score.is_mate():
                cp = 10000 if score.mate() > 0 else -10000
            else:
                cp = score.score()
            top_evals.append(cp)

        best_move = top_moves[0] if top_moves else None
        best_eval = top_evals[0] if top_evals else 0
        position_type = self.get_position_type(best_eval, top_evals)

        # Check if position is sharp (multiple good moves with similar eval)
        is_sharp = False
        if len(top_evals) >= 3:
            eval_spread = max(top_evals[:3]) - min(top_evals[:3])
            is_sharp = eval_spread < 30  # If top 3 moves within 30 CP

        results = []
        all_moves = []  # For consistency checking

        # Generate multiple moves if requested
        for gen_idx in range(self.num_generations):
            response, inference_time, response_tokens = self.generate_move(fen)
            model_move = self.extract_move_from_response(response)
            has_correct_format = self.has_correct_format(response)
            all_moves.append(model_move)

            # Evaluate the move
            is_legal = False
            is_best = False
            is_top3 = False
            is_top5 = False
            centipawn_loss = None
            move_eval = None
            move_quality = "illegal"

            # Check for draw to loss conversion
            draw_to_loss = False
            win_preserved = None

            if model_move:
                try:
                    # Check if move is legal
                    move_obj = board.parse_san(model_move)
                    is_legal = True

                    # Get evaluation after this move
                    board.push(move_obj)
                    result = self.engine.analyse(board, chess.engine.Limit(depth=20))
                    score = result["score"].relative

                    # Convert to centipawns (from opponent's perspective, so negate)
                    if score.is_mate():
                        move_eval = -10000 if score.mate() > 0 else 10000
                    else:
                        move_eval = -score.score()

                    board.pop()

                    # Calculate centipawn loss
                    centipawn_loss = best_eval - move_eval
                    move_quality = self.classify_move_quality(centipawn_loss)

                    # Check if it's in top moves
                    is_best = (model_move == best_move)
                    is_top3 = (model_move in top_moves[:3])
                    is_top5 = (model_move in top_moves[:5])

                    # Check draw to loss conversion
                    if abs(best_eval) < 50:  # Position was roughly equal
                        if move_eval < -150:  # Move leads to losing position
                            draw_to_loss = True

                    # Check win preservation
                    if best_eval > 300:  # Position was winning
                        win_preserved = move_eval > 200  # Still winning after move

                except (chess.IllegalMoveError, chess.InvalidMoveError, chess.AmbiguousMoveError):
                    pass

            results.append({
                'fen': fen,
                'generation_idx': gen_idx,
                'model_move': model_move,
                'best_move': best_move,
                'top_3_moves': ','.join(top_moves[:3]) if len(top_moves) >= 3 else ','.join(top_moves),
                'top_5_moves': ','.join(top_moves[:5]) if len(top_moves) >= 5 else ','.join(top_moves),
                'is_legal': is_legal,
                'is_best': is_best,
                'is_top3': is_top3,
                'is_top5': is_top5,
                'best_eval': best_eval,
                'move_eval': move_eval,
                'centipawn_loss': centipawn_loss,
                'move_quality': move_quality,
                'game_phase': phase,
                'move_number': move_number,
                'material_count': material,
                'is_tactical': is_tactical,
                'is_sharp': is_sharp,
                'position_type': position_type,
                'draw_to_loss': draw_to_loss,
                'win_preserved': win_preserved,
                'correct_format': has_correct_format,
                'inference_time': inference_time,
                'response_tokens': response_tokens,
                'ground_truth_eval': ground_truth_eval,
                'full_response': response
            })

        # Calculate consistency metrics (only if multiple generations)
        if self.num_generations > 1:
            legal_moves = [m for m in all_moves if m is not None]
            if len(legal_moves) > 1:
                most_common = max(set(legal_moves), key=legal_moves.count)
                consistency_rate = legal_moves.count(most_common) / len(legal_moves)
            else:
                consistency_rate = 0.0

            # Add consistency to all results for this position
            for r in results:
                r['move_consistency'] = consistency_rate

        return results

    def evaluate_dataset(self, dataset, num_samples=None, output_path=None):
        """
        Evaluate model on a dataset of positions.

        Args:
            dataset: HuggingFace dataset with 'FEN' column
            num_samples: Number of samples to evaluate (None = all)
            output_path: Path to save CSV results
        """
        if num_samples:
            dataset = dataset.select(range(min(num_samples, len(dataset))))

        all_results = []

        print(f"Evaluating {len(dataset)} positions...")
        for idx, example in enumerate(tqdm(dataset)):
            fen = example['FEN']
            ground_truth = example.get('Evaluation', None)

            try:
                position_results = self.evaluate_position(fen, ground_truth)
                all_results.extend(position_results)
            except Exception as e:
                print(f"Error on position {idx}: {e}")
                continue

        # Create DataFrame
        df = pd.DataFrame(all_results)

        # Save to CSV
        if output_path is None:
            output_path = f"{self.model_name}_results.csv"

        df.to_csv(output_path, index=False)
        print(f"Results saved to {output_path}")

        # Print summary statistics
        self.print_summary(df)

        return df

    def print_summary(self, df):
        """Print summary statistics."""
        print(f"\n{'='*60}")
        print(f"Summary for {self.model_name}")
        print(f"{'='*60}")
        print(f"Total positions: {len(df[df['generation_idx'] == 0])}")
        print(f"Total generations: {len(df)}")

        print(f"\n=== CORE METRICS ===")
        print(f"Legal move rate: {df['is_legal'].mean()*100:.2f}%")
        print(f"Best move rate: {df['is_best'].mean()*100:.2f}%")
        print(f"Top-3 move rate: {df['is_top3'].mean()*100:.2f}%")
        print(f"Top-5 move rate: {df['is_top5'].mean()*100:.2f}%")
        print(f"Format compliance: {df['correct_format'].mean()*100:.2f}%")

        legal_moves = df[df['is_legal']]
        if len(legal_moves) > 0:
            print(f"\n=== CENTIPAWN METRICS ===")
            print(f"Avg centipawn loss: {legal_moves['centipawn_loss'].mean():.2f}")
            print(f"Median centipawn loss: {legal_moves['centipawn_loss'].median():.2f}")
            print(f"Std dev centipawn loss: {legal_moves['centipawn_loss'].std():.2f}")

            print(f"\n=== MOVE QUALITY BREAKDOWN ===")
            for quality in ['excellent', 'good', 'inaccuracy', 'mistake', 'blunder']:
                count = (legal_moves['move_quality'] == quality).sum()
                pct = count / len(legal_moves) * 100
                print(f"{quality.capitalize()}: {pct:.2f}% ({count}/{len(legal_moves)})")

        print(f"\n=== POSITION TYPE PERFORMANCE ===")
        for pos_type in ['winning', 'advantage', 'equal', 'losing']:
            type_df = df[df['position_type'] == pos_type]
            if len(type_df) > 0:
                print(f"{pos_type.capitalize()}: Legal={type_df['is_legal'].mean()*100:.1f}%, Best={type_df['is_best'].mean()*100:.1f}%")

        # Draw to loss conversion
        draw_positions = df[df['position_type'] == 'equal']
        if len(draw_positions) > 0:
            draw_to_loss_rate = draw_positions['draw_to_loss'].sum() / len(draw_positions) * 100
            print(f"\nDraw to loss conversion rate: {draw_to_loss_rate:.2f}%")

        # Win preservation
        win_positions = df[df['position_type'] == 'winning']
        if len(win_positions) > 0:
            win_preserved = win_positions['win_preserved'].sum() / len(win_positions) * 100
            print(f"Win preservation rate: {win_preserved:.2f}%")

        print(f"\n=== GAME PHASE PERFORMANCE ===")
        for phase in ['opening', 'middlegame', 'endgame']:
            phase_df = df[df['game_phase'] == phase]
            if len(phase_df) > 0:
                print(f"{phase.capitalize()}: Legal={phase_df['is_legal'].mean()*100:.1f}%, Best={phase_df['is_best'].mean()*100:.1f}%", end="")
                legal_phase = phase_df[phase_df['is_legal']]
                if len(legal_phase) > 0:
                    print(f", CP Loss={legal_phase['centipawn_loss'].mean():.1f}")
                else:
                    print()

        print(f"\n=== TACTICAL VS POSITIONAL ===")
        tactical_df = df[df['is_tactical']]
        positional_df = df[~df['is_tactical']]
        if len(tactical_df) > 0:
            print(f"Tactical: Legal={tactical_df['is_legal'].mean()*100:.1f}%, Best={tactical_df['is_best'].mean()*100:.1f}%")
        if len(positional_df) > 0:
            print(f"Positional: Legal={positional_df['is_legal'].mean()*100:.1f}%, Best={positional_df['is_best'].mean()*100:.1f}%")

        print(f"\n=== SHARP VS QUIET ===")
        sharp_df = df[df['is_sharp']]
        quiet_df = df[~df['is_sharp']]
        if len(sharp_df) > 0:
            print(f"Sharp: Legal={sharp_df['is_legal'].mean()*100:.1f}%, Best={sharp_df['is_best'].mean()*100:.1f}%")
        if len(quiet_df) > 0:
            print(f"Quiet: Legal={quiet_df['is_legal'].mean()*100:.1f}%, Best={quiet_df['is_best'].mean()*100:.1f}%")

        print(f"\n=== EFFICIENCY METRICS ===")
        print(f"Avg inference time: {df['inference_time'].mean():.3f}s")
        print(f"Avg response tokens: {df['response_tokens'].mean():.1f}")

        if self.num_generations > 1 and 'move_consistency' in df.columns:
            print(f"\n=== CONSISTENCY ===")
            print(f"Avg move consistency: {df['move_consistency'].mean()*100:.1f}%")

        print(f"{'='*60}\n")

    def cleanup(self):
        """Cleanup resources."""
        self.engine.quit()


# Example usage
if __name__ == "__main__":
    # Configuration
    MODEL_PATH = "czovekboti/stf_model"  # Change this for different models
    MODEL_NAME = "sft_model"  # Change this for naming output files
    NUM_SAMPLES = 3  # Number of positions to evaluate (None for all)
    NUM_GENERATIONS = 2  # Number of generations per position

    # Load dataset
    print("Loading dataset...")
    dataset = load_dataset("czovekboti/chessdata", split="train")

    # Option 1: Load model once and reuse it
    print("Loading model (will be reused)...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_PATH,
        max_seq_length=1024,
        load_in_4bit=False,
        fast_inference=False,
        max_lora_rank=32,
        gpu_memory_utilization=0.8,
        resize_model_vocab=151669,
    )
    FastLanguageModel.for_inference(model)

    # Initialize evaluator with pre-loaded model
    evaluator = ChessModelEvaluator(
        model_path=MODEL_PATH,  # Not used when model is provided
        model_name=MODEL_NAME,
        stockfish_path="/usr/games/stockfish",  # Adjust if needed
        num_generations=NUM_GENERATIONS,
        model=model,  # Pass pre-loaded model
        tokenizer=tokenizer  # Pass pre-loaded tokenizer
    )

    # Run evaluation
    results_df = evaluator.evaluate_dataset(
        dataset=dataset,
        num_samples=NUM_SAMPLES,
        output_path=f"{MODEL_NAME}_results.csv"
    )

    # Cleanup
    evaluator.cleanup()

    print("Evaluation complete!")

    # Option 2: If you want to evaluate multiple model variants without reloading base model
    # You can create multiple evaluators with the same model and tokenizer:
    # evaluator2 = ChessModelEvaluator(
    #     model_path=None,
    #     model_name="same_model_variant2",
    #     num_generations=3,
    #     model=model,
    #     tokenizer=tokenizer
    # )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading dataset...
Loading model (will be reused)...
==((====))==  Unsloth 2025.12.1: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.12.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


Using pre-loaded model for: sft_model
Initializing Stockfish from: /usr/games/stockfish
Evaluating 3 positions...


100%|██████████| 3/3 [01:19<00:00, 26.33s/it]

Results saved to sft_model_results.csv

Summary for sft_model
Total positions: 3
Total generations: 6

=== CORE METRICS ===
Legal move rate: 33.33%
Best move rate: 16.67%
Top-3 move rate: 16.67%
Top-5 move rate: 16.67%
Format compliance: 0.00%

=== CENTIPAWN METRICS ===
Avg centipawn loss: 20.50
Median centipawn loss: 20.50
Std dev centipawn loss: 23.33

=== MOVE QUALITY BREAKDOWN ===
Excellent: 50.00% (1/2)
Good: 50.00% (1/2)
Inaccuracy: 0.00% (0/2)
Mistake: 0.00% (0/2)
Blunder: 0.00% (0/2)

=== POSITION TYPE PERFORMANCE ===
Advantage: Legal=0.0%, Best=0.0%
Equal: Legal=50.0%, Best=25.0%

Draw to loss conversion rate: 0.00%

=== GAME PHASE PERFORMANCE ===
Opening: Legal=33.3%, Best=16.7%, CP Loss=20.5

=== TACTICAL VS POSITIONAL ===
Positional: Legal=33.3%, Best=16.7%

=== SHARP VS QUIET ===
Sharp: Legal=50.0%, Best=0.0%
Quiet: Legal=25.0%, Best=25.0%

=== EFFICIENCY METRICS ===
Avg inference time: 6.579s
Avg response tokens: 64.3

=== CONSISTENCY ===
Avg move consistency: 50.0%

Eval

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
import glob
from scipy import stats

class ChessModelComparison:
    def __init__(self, csv_files):
        """
        Initialize comparison with multiple model result CSVs.

        Args:
            csv_files: List of paths to CSV files or a pattern (e.g., "*_results.csv")
        """
        if isinstance(csv_files, str):
            csv_files = glob.glob(csv_files)

        self.dataframes = {}
        self.model_names = []

        for csv_file in csv_files:
            model_name = Path(csv_file).stem.replace('_results', '')
            df = pd.read_csv(csv_file)
            self.dataframes[model_name] = df
            self.model_names.append(model_name)

        print(f"Loaded {len(self.model_names)} models: {', '.join(self.model_names)}")

        # Set style
        sns.set_style("whitegrid")
        plt.rcParams['figure.figsize'] = (12, 6)
        self.colors = sns.color_palette("husl", len(self.model_names))

    def create_summary_table(self):
        """Create a comprehensive summary table comparing all models."""
        summary_data = []

        for model_name in self.model_names:
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal']]

            # Basic metrics
            row = {
                'Model': model_name,
                'Positions': len(df[df['generation_idx'] == 0]),
                'Legal (%)': df['is_legal'].mean() * 100,
                'Best (%)': df['is_best'].mean() * 100,
                'Top-3 (%)': df['is_top3'].mean() * 100,
                'Top-5 (%)': df['is_top5'].mean() * 100,
                'Format (%)': df['correct_format'].mean() * 100,
            }

            if len(legal_df) > 0:
                row['Avg CP Loss'] = legal_df['centipawn_loss'].mean()
                row['Med CP Loss'] = legal_df['centipawn_loss'].median()
                row['Std CP Loss'] = legal_df['centipawn_loss'].std()

                # Move quality
                row['Excellent (%)'] = (legal_df['move_quality'] == 'excellent').mean() * 100
                row['Blunder (%)'] = (legal_df['move_quality'] == 'blunder').mean() * 100
                row['Mistake (%)'] = (legal_df['move_quality'] == 'mistake').mean() * 100
                row['Inaccuracy (%)'] = (legal_df['move_quality'] == 'inaccuracy').mean() * 100

            # Draw to loss
            draw_df = df[df['position_type'] == 'equal']
            if len(draw_df) > 0:
                row['Draw→Loss (%)'] = draw_df['draw_to_loss'].sum() / len(draw_df) * 100

            # Win preservation
            win_df = df[df['position_type'] == 'winning']
            if len(win_df) > 0:
                row['Win Kept (%)'] = win_df['win_preserved'].sum() / len(win_df) * 100

            # Efficiency
            row['Avg Time (s)'] = df['inference_time'].mean()
            row['Avg Tokens'] = df['response_tokens'].mean()

            summary_data.append(row)

        summary_df = pd.DataFrame(summary_data)
        print("\n" + "="*100)
        print("COMPREHENSIVE MODEL COMPARISON")
        print("="*100)
        print(summary_df.to_string(index=False, float_format='%.2f'))
        print("="*100 + "\n")

        return summary_df

    def plot_core_metrics(self, save_path='core_metrics.png'):
        """Plot core performance metrics."""
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))

        metrics = [
            ('Legal (%)', 'is_legal', 100),
            ('Best (%)', 'is_best', 100),
            ('Top-3 (%)', 'is_top3', 100),
            ('Top-5 (%)', 'is_top5', 100),
            ('Format (%)', 'correct_format', 100),
            ('Avg CP Loss', 'centipawn_loss', 1)
        ]

        for idx, (label, col, multiplier) in enumerate(metrics):
            ax = axes[idx // 3, idx % 3]
            data = []
            labels = []

            for model_name in self.model_names:
                df = self.dataframes[model_name]
                if col == 'centipawn_loss':
                    legal_df = df[df['is_legal']]
                    value = legal_df[col].mean() if len(legal_df) > 0 else 0
                else:
                    value = df[col].mean() * multiplier

                data.append(value)
                labels.append(model_name)

            bars = ax.bar(labels, data, color=self.colors, alpha=0.8)
            ax.set_ylabel(label)
            ax.set_title(label, fontsize=12, fontweight='bold')

            if col != 'centipawn_loss':
                ax.set_ylim([0, 100])

            # Add value labels
            for i, (bar, v) in enumerate(zip(bars, data)):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{v:.1f}', ha='center', va='bottom', fontsize=10)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_move_quality_breakdown(self, save_path='move_quality_breakdown.png'):
        """Plot detailed move quality breakdown."""
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        qualities = ['excellent', 'good', 'inaccuracy', 'mistake', 'blunder']
        quality_colors = ['#2ecc71', '#3498db', '#f39c12', '#e67e22', '#e74c3c']

        # Stacked bar chart
        x = np.arange(len(self.model_names))
        width = 0.6
        bottom = np.zeros(len(self.model_names))

        for quality, color in zip(qualities, quality_colors):
            values = []
            for model_name in self.model_names:
                df = self.dataframes[model_name]
                legal_df = df[df['is_legal']]
                if len(legal_df) > 0:
                    pct = (legal_df['move_quality'] == quality).mean() * 100
                    values.append(pct)
                else:
                    values.append(0)

            axes[0].bar(x, values, width, bottom=bottom, label=quality.capitalize(),
                       color=color, alpha=0.8)
            bottom += values

        axes[0].set_ylabel('Percentage (%)')
        axes[0].set_title('Move Quality Distribution (Stacked)', fontsize=12, fontweight='bold')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(self.model_names)
        axes[0].legend(loc='upper right')
        axes[0].set_ylim([0, 100])

        # Grouped bar for key categories
        key_categories = ['Excellent', 'Blunder', 'Mistake']
        x = np.arange(len(key_categories))
        width = 0.8 / len(self.model_names)

        for i, model_name in enumerate(self.model_names):
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal']]

            if len(legal_df) > 0:
                excellent = (legal_df['move_quality'] == 'excellent').mean() * 100
                blunder = (legal_df['move_quality'] == 'blunder').mean() * 100
                mistake = (legal_df['move_quality'] == 'mistake').mean() * 100
                values = [excellent, blunder, mistake]
            else:
                values = [0, 0, 0]

            offset = (i - len(self.model_names)/2 + 0.5) * width
            axes[1].bar(x + offset, values, width, label=model_name,
                       color=self.colors[i], alpha=0.8)

        axes[1].set_ylabel('Percentage (%)')
        axes[1].set_title('Key Move Categories', fontsize=12, fontweight='bold')
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(key_categories)
        axes[1].legend()

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_centipawn_analysis(self, save_path='centipawn_analysis.png'):
        """Comprehensive centipawn loss analysis."""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # 1. CP Loss distribution
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal']]
            if len(legal_df) > 0:
                cp_loss = legal_df['centipawn_loss'].clip(0, 500)
                axes[0, 0].hist(cp_loss, bins=50, alpha=0.5, label=model_name, density=True)

        axes[0, 0].set_xlabel('Centipawn Loss')
        axes[0, 0].set_ylabel('Density')
        axes[0, 0].set_title('CP Loss Distribution', fontsize=12, fontweight='bold')
        axes[0, 0].legend()
        axes[0, 0].axvline(x=0, color='red', linestyle='--', alpha=0.3)

        # 2. Cumulative CP Loss
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal']]
            if len(legal_df) > 0:
                sorted_cp = np.sort(legal_df['centipawn_loss'])
                cumulative = np.arange(1, len(sorted_cp) + 1) / len(sorted_cp) * 100
                axes[0, 1].plot(sorted_cp, cumulative, label=model_name, linewidth=2)

        axes[0, 1].set_xlabel('Centipawn Loss')
        axes[0, 1].set_ylabel('Cumulative Percentage (%)')
        axes[0, 1].set_title('Cumulative CP Loss', fontsize=12, fontweight='bold')
        axes[0, 1].set_xlim([0, 300])
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

        # 3. CP Loss ranges
        bins = [0, 50, 100, 200, 500, float('inf')]
        labels = ['0-50', '50-100', '100-200', '200-500', '500+']

        for model_name in self.model_names:
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal']]
            if len(legal_df) > 0:
                binned = pd.cut(legal_df['centipawn_loss'], bins=bins, labels=labels)
                counts = binned.value_counts().sort_index()
                percentages = counts / len(legal_df) * 100
                axes[1, 0].plot(labels, percentages, marker='o', label=model_name,
                              linewidth=2, markersize=8)

        axes[1, 0].set_xlabel('CP Loss Range')
        axes[1, 0].set_ylabel('Percentage of Moves (%)')
        axes[1, 0].set_title('Move Distribution by CP Loss Range', fontweight='bold')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)

        # 4. Box plot comparison
        data_for_box = []
        labels_for_box = []
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal']]
            if len(legal_df) > 0:
                cp_clipped = legal_df['centipawn_loss'].clip(0, 500)
                data_for_box.append(cp_clipped)
                labels_for_box.append(model_name)

        bp = axes[1, 1].boxplot(data_for_box, labels=labels_for_box, patch_artist=True)
        for patch, color in zip(bp['boxes'], self.colors):
            patch.set_facecolor(color)
            patch.set_alpha(0.6)

        axes[1, 1].set_ylabel('Centipawn Loss')
        axes[1, 1].set_title('CP Loss Distribution (Box Plot)', fontweight='bold')
        axes[1, 1].grid(True, alpha=0.3, axis='y')

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_phase_analysis(self, save_path='phase_analysis.png'):
        """Analyze performance across game phases."""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        phases = ['opening', 'middlegame', 'endgame']
        x = np.arange(len(phases))
        width = 0.8 / len(self.model_names)

        # Legal rate by phase
        for i, model_name in enumerate(self.model_names):
            df = self.dataframes[model_name]
            values = []
            for phase in phases:
                phase_df = df[df['game_phase'] == phase]
                values.append(phase_df['is_legal'].mean() * 100 if len(phase_df) > 0 else 0)

            offset = (i - len(self.model_names)/2 + 0.5) * width
            axes[0, 0].bar(x + offset, values, width, label=model_name,
                          color=self.colors[i], alpha=0.8)

        axes[0, 0].set_ylabel('Legal Rate (%)')
        axes[0, 0].set_title('Legal Move Rate by Phase', fontweight='bold')
        axes[0, 0].set_xticks(x)
        axes[0, 0].set_xticklabels([p.capitalize() for p in phases])
        axes[0, 0].legend()
        axes[0, 0].set_ylim([0, 100])

        # Best move rate by phase
        for i, model_name in enumerate(self.model_names):
            df = self.dataframes[model_name]
            values = []
            for phase in phases:
                phase_df = df[df['game_phase'] == phase]
                values.append(phase_df['is_best'].mean() * 100 if len(phase_df) > 0 else 0)

            offset = (i - len(self.model_names)/2 + 0.5) * width
            axes[0, 1].bar(x + offset, values, width, label=model_name,
                          color=self.colors[i], alpha=0.8)

        axes[0, 1].set_ylabel('Best Move Rate (%)')
        axes[0, 1].set_title('Best Move Rate by Phase', fontweight='bold')
        axes[0, 1].set_xticks(x)
        axes[0, 1].set_xticklabels([p.capitalize() for p in phases])
        axes[0, 1].legend()
        axes[0, 1].set_ylim([0, 100])

        # CP Loss by phase
        for i, model_name in enumerate(self.model_names):
            df = self.dataframes[model_name]
            values = []
            for phase in phases:
                phase_df = df[(df['game_phase'] == phase) & (df['is_legal'])]
                values.append(phase_df['centipawn_loss'].mean() if len(phase_df) > 0 else 0)

            offset = (i - len(self.model_names)/2 + 0.5) * width
            axes[1, 0].bar(x + offset, values, width, label=model_name,
                          color=self.colors[i], alpha=0.8)

        axes[1, 0].set_ylabel('Average CP Loss')
        axes[1, 0].set_title('CP Loss by Phase', fontweight='bold')
        axes[1, 0].set_xticks(x)
        axes[1, 0].set_xticklabels([p.capitalize() for p in phases])
        axes[1, 0].legend()

        # Performance by move number
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            # Group by move number ranges
            df['move_range'] = pd.cut(df['move_number'], bins=[0, 10, 20, 30, 40, 100],
                                     labels=['1-10', '11-20', '21-30', '31-40', '40+'])
            grouped = df[df['is_legal']].groupby('move_range')['is_best'].mean() * 100
            axes[1, 1].plot(range(len(grouped)), grouped.values, marker='o',
                          label=model_name, linewidth=2, markersize=8)

        axes[1, 1].set_xlabel('Move Number Range')
        axes[1, 1].set_ylabel('Best Move Rate (%)')
        axes[1, 1].set_title('Performance by Move Number', fontweight='bold')
        axes[1, 1].set_xticks(range(5))
        axes[1, 1].set_xticklabels(['1-10', '11-20', '21-30', '31-40', '40+'])
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_position_type_analysis(self, save_path='position_type_analysis.png'):
        """Analyze performance by position type."""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        pos_types = ['winning', 'advantage', 'equal', 'losing']
        x = np.arange(len(pos_types))
        width = 0.8 / len(self.model_names)

        # Legal rate by position type
        for i, model_name in enumerate(self.model_names):
            df = self.dataframes[model_name]
            values = []
            for pos_type in pos_types:
                type_df = df[df['position_type'] == pos_type]
                values.append(type_df['is_legal'].mean() * 100 if len(type_df) > 0 else 0)

            offset = (i - len(self.model_names)/2 + 0.5) * width
            axes[0, 0].bar(x + offset, values, width, label=model_name,
                          color=self.colors[i], alpha=0.8)

        axes[0, 0].set_ylabel('Legal Rate (%)')
        axes[0, 0].set_title('Legal Rate by Position Type', fontweight='bold')
        axes[0, 0].set_xticks(x)
        axes[0, 0].set_xticklabels([p.capitalize() for p in pos_types], rotation=15)
        axes[0, 0].legend()
        axes[0, 0].set_ylim([0, 110])

        # Best move rate
        for i, model_name in enumerate(self.model_names):
            df = self.dataframes[model_name]
            values = []
            for pos_type in pos_types:
                type_df = df[df['position_type'] == pos_type]
                values.append(type_df['is_best'].mean() * 100 if len(type_df) > 0 else 0)

            offset = (i - len(self.model_names)/2 + 0.5) * width
            axes[0, 1].bar(x + offset, values, width, label=model_name,
                          color=self.colors[i], alpha=0.8)

        axes[0, 1].set_ylabel('Best Move Rate (%)')
        axes[0, 1].set_title('Best Move Rate by Position Type', fontweight='bold')
        axes[0, 1].set_xticks(x)
        axes[0, 1].set_xticklabels([p.capitalize() for p in pos_types], rotation=15)
        axes[0, 1].legend()
        axes[0, 1].set_ylim([0, 110])

        # Draw to loss conversion
        data = []
        labels = []
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            draw_df = df[df['position_type'] == 'equal']
            if len(draw_df) > 0:
                rate = draw_df['draw_to_loss'].sum() / len(draw_df) * 100
                data.append(rate)
                labels.append(model_name)

        if data:
            axes[1, 0].bar(labels, data, color=self.colors[:len(labels)], alpha=0.8)
            axes[1, 0].set_ylabel('Draw to Loss Rate (%)')
            axes[1, 0].set_title('Draw to Loss Conversion Rate', fontweight='bold')
            axes[1, 0].set_ylim([0, max(data) * 1.2 if data else 100])
            for i, v in enumerate(data):
                axes[1, 0].text(i, v + max(data)*0.02, f'{v:.1f}%', ha='center', fontsize=10)
        else:
            axes[1, 0].text(0.5, 0.5, 'No equal positions in sample',
                          ha='center', va='center', transform=axes[1, 0].transAxes,
                          fontsize=12, style='italic', color='gray')
            axes[1, 0].set_title('Draw to Loss Conversion Rate', fontweight='bold')

        # Win preservation
        data = []
        labels = []
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            win_df = df[df['position_type'] == 'winning']
            if len(win_df) > 0:
                rate = win_df['win_preserved'].sum() / len(win_df) * 100
                data.append(rate)
                labels.append(model_name)

        if data:
            axes[1, 1].bar(labels, data, color=self.colors[:len(labels)], alpha=0.8)
            axes[1, 1].set_ylabel('Win Preservation Rate (%)')
            axes[1, 1].set_title('Win Preservation Rate', fontweight='bold')
            axes[1, 1].set_ylim([0, 110])
            for i, v in enumerate(data):
                axes[1, 1].text(i, v + 2, f'{v:.1f}%', ha='center', fontsize=10)
        else:
            axes[1, 1].text(0.5, 0.5, 'No winning positions in sample',
                          ha='center', va='center', transform=axes[1, 1].transAxes,
                          fontsize=12, style='italic', color='gray')
            axes[1, 1].set_title('Win Preservation Rate', fontweight='bold')

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_tactical_positional(self, save_path='tactical_positional.png'):
        """Compare tactical vs positional performance."""
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        categories = ['Tactical', 'Positional']
        x = np.arange(len(categories))
        width = 0.8 / len(self.model_names)

        # Legal rate
        for i, model_name in enumerate(self.model_names):
            df = self.dataframes[model_name]
            tactical = df[df['is_tactical']]['is_legal'].mean() * 100
            positional = df[~df['is_tactical']]['is_legal'].mean() * 100
            values = [tactical, positional]

            offset = (i - len(self.model_names)/2 + 0.5) * width
            axes[0, 0].bar(x + offset, values, width, label=model_name,
                          color=self.colors[i], alpha=0.8)

        axes[0, 0].set_ylabel('Legal Rate (%)')
        axes[0, 0].set_title('Legal Rate: Tactical vs Positional', fontweight='bold')
        axes[0, 0].set_xticks(x)
        axes[0, 0].set_xticklabels(categories)
        axes[0, 0].legend()
        axes[0, 0].set_ylim([0, 100])

        # Best move rate
        for i, model_name in enumerate(self.model_names):
            df = self.dataframes[model_name]
            tactical = df[df['is_tactical']]['is_best'].mean() * 100
            positional = df[~df['is_tactical']]['is_best'].mean() * 100
            values = [tactical, positional]

            offset = (i - len(self.model_names)/2 + 0.5) * width
            axes[0, 1].bar(x + offset, values, width, label=model_name,
                          color=self.colors[i], alpha=0.8)

        axes[0, 1].set_ylabel('Best Move Rate (%)')
        axes[0, 1].set_title('Best Move: Tactical vs Positional', fontweight='bold')
        axes[0, 1].set_xticks(x)
        axes[0, 1].set_xticklabels(categories)
        axes[0, 1].legend()
        axes[0, 1].set_ylim([0, 100])

        # Sharp vs Quiet positions
        categories = ['Sharp', 'Quiet']
        x = np.arange(len(categories))

        for i, model_name in enumerate(self.model_names):
            df = self.dataframes[model_name]
            sharp = df[df['is_sharp']]['is_best'].mean() * 100
            quiet = df[~df['is_sharp']]['is_best'].mean() * 100
            values = [sharp, quiet]

            offset = (i - len(self.model_names)/2 + 0.5) * width
            axes[1, 0].bar(x + offset, values, width, label=model_name,
                          color=self.colors[i], alpha=0.8)

        axes[1, 0].set_ylabel('Best Move Rate (%)')
        axes[1, 0].set_title('Best Move: Sharp vs Quiet Positions', fontweight='bold')
        axes[1, 0].set_xticks(x)
        axes[1, 0].set_xticklabels(categories)
        axes[1, 0].legend()
        axes[1, 0].set_ylim([0, 100])

        # Performance by material count
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            df['material_range'] = pd.cut(df['material_count'],
                                         bins=[0, 20, 40, 60, 80, 100],
                                         labels=['0-20', '20-40', '40-60', '60-80', '80+'])
            grouped = df[df['is_legal']].groupby('material_range')['is_best'].mean() * 100
            axes[1, 1].plot(range(len(grouped)), grouped.values, marker='o',
                          label=model_name, linewidth=2, markersize=8)

        axes[1, 1].set_xlabel('Material Count')
        axes[1, 1].set_ylabel('Best Move Rate (%)')
        axes[1, 1].set_title('Performance by Material Count', fontweight='bold')
        axes[1, 1].set_xticks(range(5))
        axes[1, 1].set_xticklabels(['0-20', '20-40', '40-60', '60-80', '80+'])
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_efficiency_metrics(self, save_path='efficiency_metrics.png'):
        """Plot efficiency and consistency metrics."""
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        # Inference time
        data = []
        labels = []
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            data.append(df['inference_time'].mean())
            labels.append(model_name)

        axes[0, 0].bar(labels, data, color=self.colors, alpha=0.8)
        axes[0, 0].set_ylabel('Time (seconds)')
        axes[0, 0].set_title('Average Inference Time', fontweight='bold')
        for i, v in enumerate(data):
            axes[0, 0].text(i, v + max(data)*0.02, f'{v:.3f}s', ha='center', fontsize=10)

        # Response tokens
        data = []
        labels = []
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            data.append(df['response_tokens'].mean())
            labels.append(model_name)

        axes[0, 1].bar(labels, data, color=self.colors, alpha=0.8)
        axes[0, 1].set_ylabel('Token Count')
        axes[0, 1].set_title('Average Response Length (tokens)', fontweight='bold')
        for i, v in enumerate(data):
            axes[0, 1].text(i, v + max(data)*0.02, f'{v:.1f}', ha='center', fontsize=10)

        # Tokens per second
        data = []
        labels = []
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            tps = df['response_tokens'].mean() / df['inference_time'].mean()
            data.append(tps)
            labels.append(model_name)

        axes[1, 0].bar(labels, data, color=self.colors, alpha=0.8)
        axes[1, 0].set_ylabel('Tokens/Second')
        axes[1, 0].set_title('Generation Speed', fontweight='bold')
        for i, v in enumerate(data):
            axes[1, 0].text(i, v + max(data)*0.02, f'{v:.1f}', ha='center', fontsize=10)

        # Move consistency (if available)
        if 'move_consistency' in self.dataframes[self.model_names[0]].columns:
            data = []
            labels = []
            for model_name in self.model_names:
                df = self.dataframes[model_name]
                if df['move_consistency'].notna().any():
                    data.append(df['move_consistency'].mean() * 100)
                    labels.append(model_name)

            if data:
                axes[1, 1].bar(labels, data, color=self.colors[:len(labels)], alpha=0.8)
                axes[1, 1].set_ylabel('Consistency (%)')
                axes[1, 1].set_title('Move Consistency (Multi-generation)', fontweight='bold')
                axes[1, 1].set_ylim([0, 100])
                for i, v in enumerate(data):
                    axes[1, 1].text(i, v + 2, f'{v:.1f}%', ha='center', fontsize=10)
            else:
                axes[1, 1].text(0.5, 0.5, 'No multi-generation data',
                              ha='center', va='center', transform=axes[1, 1].transAxes,
                              fontsize=12)
                axes[1, 1].set_xticks([])
                axes[1, 1].set_yticks([])
        else:
            axes[1, 1].text(0.5, 0.5, 'No consistency data available',
                          ha='center', va='center', transform=axes[1, 1].transAxes,
                          fontsize=12)
            axes[1, 1].set_xticks([])
            axes[1, 1].set_yticks([])

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_radar_comparison(self, save_path='radar_comparison.png'):
        """Create radar chart for multi-dimensional comparison."""
        from math import pi

        categories = ['Legal\nRate', 'Best\nMove', 'Top-3', 'Low CP\nLoss',
                     'Win\nPreserve', 'Format']
        N = len(categories)

        fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]

        for i, model_name in enumerate(self.model_names):
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal']]
            win_df = df[df['position_type'] == 'winning']

            values = [
                df['is_legal'].mean() * 100,
                df['is_best'].mean() * 100,
                df['is_top3'].mean() * 100,
                100 - (legal_df['centipawn_loss'].mean() / 5) if len(legal_df) > 0 else 0,  # Normalized
                win_df['win_preserved'].mean() * 100 if len(win_df) > 0 else 0,
                df['correct_format'].mean() * 100
            ]
            values += values[:1]

            ax.plot(angles, values, 'o-', linewidth=2, label=model_name,
                   color=self.colors[i])
            ax.fill(angles, values, alpha=0.15, color=self.colors[i])

        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(categories, size=10)
        ax.set_ylim(0, 100)
        ax.set_title('Multi-Dimensional Performance Comparison',
                    size=14, fontweight='bold', pad=20)
        ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
        ax.grid(True)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_agreement_matrix(self, save_path='agreement_matrix.png'):
        """Create agreement matrix showing how often models agree."""
        n_models = len(self.model_names)
        agreement_matrix = np.zeros((n_models, n_models))

        # Get all positions from first model
        first_df = self.dataframes[self.model_names[0]]
        positions = first_df[first_df['generation_idx'] == 0]['fen'].unique()

        for i, model1 in enumerate(self.model_names):
            for j, model2 in enumerate(self.model_names):
                if i == j:
                    agreement_matrix[i, j] = 100
                    continue

                df1 = self.dataframes[model1]
                df2 = self.dataframes[model2]

                agreements = 0
                total = 0

                for fen in positions:
                    move1 = df1[(df1['fen'] == fen) & (df1['generation_idx'] == 0)]['model_move'].values
                    move2 = df2[(df2['fen'] == fen) & (df2['generation_idx'] == 0)]['model_move'].values

                    if len(move1) > 0 and len(move2) > 0:
                        if move1[0] == move2[0] and move1[0] is not None:
                            agreements += 1
                        total += 1

                agreement_matrix[i, j] = (agreements / total * 100) if total > 0 else 0

        fig, ax = plt.subplots(figsize=(10, 8))
        im = ax.imshow(agreement_matrix, cmap='YlOrRd', aspect='auto')

        ax.set_xticks(np.arange(n_models))
        ax.set_yticks(np.arange(n_models))
        ax.set_xticklabels(self.model_names)
        ax.set_yticklabels(self.model_names)

        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

        # Add text annotations
        for i in range(n_models):
            for j in range(n_models):
                text = ax.text(j, i, f'{agreement_matrix[i, j]:.1f}%',
                             ha="center", va="center", color="black", fontsize=10)

        ax.set_title('Model Agreement Matrix\n(% of positions where models choose same move)',
                    fontsize=12, fontweight='bold', pad=15)
        fig.colorbar(im, ax=ax, label='Agreement Rate (%)')

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_elo_estimation(self, save_path='elo_estimation.png'):
        """Estimate and compare ELO ratings based on move quality."""
        fig, ax = plt.subplots(figsize=(12, 7))

        elo_estimates = []
        model_labels = []

        for model_name in self.model_names:
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal']]

            if len(legal_df) == 0:
                continue

            # Simple ELO estimation based on accuracy and CP loss
            # This is a rough heuristic
            best_move_rate = df['is_best'].mean()
            top3_rate = df['is_top3'].mean()
            avg_cp_loss = legal_df['centipawn_loss'].mean()

            # Rough formula (not scientifically rigorous)
            base_elo = 1500
            accuracy_bonus = (best_move_rate * 800 + top3_rate * 400)
            cp_penalty = min(avg_cp_loss * 2, 500)

            estimated_elo = base_elo + accuracy_bonus - cp_penalty

            elo_estimates.append(estimated_elo)
            model_labels.append(model_name)

        bars = ax.barh(model_labels, elo_estimates, color=self.colors, alpha=0.8)
        ax.set_xlabel('Estimated ELO Rating', fontsize=12)
        ax.set_title('Estimated Playing Strength (ELO)\n(Rough approximation based on move quality)',
                    fontsize=12, fontweight='bold')
        ax.axvline(x=1500, color='red', linestyle='--', alpha=0.3, label='Base (1500)')
        ax.legend()

        for i, (bar, elo) in enumerate(zip(bars, elo_estimates)):
            width = bar.get_width()
            ax.text(width + 20, bar.get_y() + bar.get_height()/2,
                   f'{elo:.0f}', ha='left', va='center', fontsize=11, fontweight='bold')

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_scatter_relationships(self, save_path='scatter_relationships.png'):
        """Create scatter plots showing relationships between variables."""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # 1. CP Loss vs Best Eval (position difficulty)
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal'] & df['best_eval'].notna()]
            if len(legal_df) > 0:
                # Sample if too many points
                if len(legal_df) > 500:
                    legal_df = legal_df.sample(500, random_state=42)
                axes[0, 0].scatter(legal_df['best_eval'], legal_df['centipawn_loss'],
                                 alpha=0.4, s=30, label=model_name)

        axes[0, 0].set_xlabel('Position Evaluation (centipawns)')
        axes[0, 0].set_ylabel('Centipawn Loss')
        axes[0, 0].set_title('CP Loss vs Position Evaluation', fontweight='bold')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].set_xlim([-500, 500])
        axes[0, 0].set_ylim([0, 300])

        # 2. Move Number vs Best Move Rate
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            # Group by move number
            move_groups = df.groupby('move_number').agg({
                'is_best': 'mean',
                'fen': 'count'
            }).reset_index()
            # Filter groups with at least 3 positions
            move_groups = move_groups[move_groups['fen'] >= 3]
            if len(move_groups) > 0:
                axes[0, 1].scatter(move_groups['move_number'],
                                 move_groups['is_best'] * 100,
                                 alpha=0.6, s=50, label=model_name)

        axes[0, 1].set_xlabel('Move Number')
        axes[0, 1].set_ylabel('Best Move Rate (%)')
        axes[0, 1].set_title('Performance vs Move Number', fontweight='bold')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].set_ylim([0, 100])

        # 3. Material Count vs CP Loss
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal']]
            # Group by material count
            material_groups = legal_df.groupby('material_count').agg({
                'centipawn_loss': 'mean',
                'fen': 'count'
            }).reset_index()
            material_groups = material_groups[material_groups['fen'] >= 3]
            if len(material_groups) > 0:
                axes[1, 0].scatter(material_groups['material_count'],
                                 material_groups['centipawn_loss'],
                                 alpha=0.6, s=50, label=model_name)

        axes[1, 0].set_xlabel('Material Count')
        axes[1, 0].set_ylabel('Average CP Loss')
        axes[1, 0].set_title('CP Loss vs Material Count', fontweight='bold')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)

        # 4. Inference Time vs Response Tokens
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            if len(df) > 500:
                df_sample = df.sample(500, random_state=42)
            else:
                df_sample = df
            axes[1, 1].scatter(df_sample['response_tokens'],
                             df_sample['inference_time'],
                             alpha=0.4, s=30, label=model_name)

        axes[1, 1].set_xlabel('Response Tokens')
        axes[1, 1].set_ylabel('Inference Time (s)')
        axes[1, 1].set_title('Generation Speed: Tokens vs Time', fontweight='bold')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def plot_time_series_trends(self, save_path='time_series_trends.png'):
        """Plot performance trends across game progression."""
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # 1. Best Move Rate by Move Number (line plot)
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            move_groups = df.groupby('move_number').agg({
                'is_best': 'mean',
                'fen': 'count'
            }).reset_index()
            move_groups = move_groups[move_groups['fen'] >= 3]
            if len(move_groups) > 0:
                move_groups = move_groups.sort_values('move_number')
                axes[0, 0].plot(move_groups['move_number'],
                              move_groups['is_best'] * 100,
                              marker='o', linewidth=2, markersize=6,
                              label=model_name, alpha=0.8)

        axes[0, 0].set_xlabel('Move Number')
        axes[0, 0].set_ylabel('Best Move Rate (%)')
        axes[0, 0].set_title('Best Move Rate Across Game', fontweight='bold')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].set_ylim([0, 100])

        # 2. CP Loss by Move Number
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            legal_df = df[df['is_legal']]
            move_groups = legal_df.groupby('move_number').agg({
                'centipawn_loss': 'mean',
                'fen': 'count'
            }).reset_index()
            move_groups = move_groups[move_groups['fen'] >= 3]
            if len(move_groups) > 0:
                move_groups = move_groups.sort_values('move_number')
                axes[0, 1].plot(move_groups['move_number'],
                              move_groups['centipawn_loss'],
                              marker='o', linewidth=2, markersize=6,
                              label=model_name, alpha=0.8)

        axes[0, 1].set_xlabel('Move Number')
        axes[0, 1].set_ylabel('Average CP Loss')
        axes[0, 1].set_title('CP Loss Across Game', fontweight='bold')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

        # 3. Legal Rate by Material Count
        for model_name in self.model_names:
            df = self.dataframes[model_name]
            material_groups = df.groupby('material_count').agg({
                'is_legal': 'mean',
                'fen': 'count'
            }).reset_index()
            material_groups = material_groups[material_groups['fen'] >= 3]
            if len(material_groups) > 0:
                material_groups = material_groups.sort_values('material_count')
                axes[1, 0].plot(material_groups['material_count'],
                              material_groups['is_legal'] * 100,
                              marker='o', linewidth=2, markersize=6,
                              label=model_name, alpha=0.8)

        axes[1, 0].set_xlabel('Material Count')
        axes[1, 0].set_ylabel('Legal Rate (%)')
        axes[1, 0].set_title('Legal Rate by Material Count', fontweight='bold')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        axes[1, 0].set_ylim([0, 100])

        # 4. Performance by Position Type (grouped line)
        pos_type_order = ['losing', 'equal', 'advantage', 'winning']
        x_pos = np.arange(len(pos_type_order))

        for model_name in self.model_names:
            df = self.dataframes[model_name]
            values = []
            for pos_type in pos_type_order:
                type_df = df[df['position_type'] == pos_type]
                if len(type_df) > 0:
                    values.append(type_df['is_best'].mean() * 100)
                else:
                    values.append(None)

            # Only plot if we have at least 2 valid points
            valid_values = [v for v in values if v is not None]
            if len(valid_values) >= 2:
                axes[1, 1].plot(x_pos, values, marker='o', linewidth=2,
                              markersize=8, label=model_name, alpha=0.8)

        axes[1, 1].set_xlabel('Position Type')
        axes[1, 1].set_ylabel('Best Move Rate (%)')
        axes[1, 1].set_title('Performance by Position Type', fontweight='bold')
        axes[1, 1].set_xticks(x_pos)
        axes[1, 1].set_xticklabels([p.capitalize() for p in pos_type_order])
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].set_ylim([0, 100])

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Saved: {save_path}")
        plt.close()

    def generate_full_report(self, output_dir='chess_analysis_results'):
        """Generate all plots and reports."""
        Path(output_dir).mkdir(exist_ok=True)

        print("\n" + "="*80)
        print("GENERATING COMPREHENSIVE COMPARATIVE ANALYSIS")
        print("="*80 + "\n")

        # Summary table
        summary_df = self.create_summary_table()
        summary_df.to_csv(f'{output_dir}/summary_comparison.csv', index=False)

        # Generate all plots
        print("\nGenerating visualizations...")
        self.plot_core_metrics(f'{output_dir}/01_core_metrics.png')
        self.plot_move_quality_breakdown(f'{output_dir}/02_move_quality.png')
        self.plot_centipawn_analysis(f'{output_dir}/03_centipawn_analysis.png')
        self.plot_phase_analysis(f'{output_dir}/04_phase_analysis.png')
        self.plot_position_type_analysis(f'{output_dir}/05_position_type.png')
        self.plot_tactical_positional(f'{output_dir}/06_tactical_positional.png')
        self.plot_efficiency_metrics(f'{output_dir}/07_efficiency.png')
        self.plot_radar_comparison(f'{output_dir}/08_radar_comparison.png')
        self.plot_agreement_matrix(f'{output_dir}/09_agreement_matrix.png')
        self.plot_elo_estimation(f'{output_dir}/10_elo_estimation.png')
        self.plot_scatter_relationships(f'{output_dir}/11_scatter_relationships.png')
        self.plot_time_series_trends(f'{output_dir}/12_time_series_trends.png')

        print(f"\n{'='*80}")
        print("✓ ANALYSIS COMPLETE!")
        print(f"{'='*80}")
        print(f"\nAll results saved to '{output_dir}/' directory")
        print("\nGenerated files:")
        print("  📊 summary_comparison.csv - Detailed metrics table")
        print("  📈 01_core_metrics.png - Core performance metrics")
        print("  📈 02_move_quality.png - Move quality breakdown")
        print("  📈 03_centipawn_analysis.png - Centipawn loss analysis")
        print("  📈 04_phase_analysis.png - Game phase performance")
        print("  📈 05_position_type.png - Position type analysis")
        print("  📈 06_tactical_positional.png - Tactical vs positional")
        print("  📈 07_efficiency.png - Speed and efficiency metrics")
        print("  📈 08_radar_comparison.png - Multi-dimensional radar chart")
        print("  📈 09_agreement_matrix.png - Model agreement heatmap")
        print("  📈 10_elo_estimation.png - Estimated playing strength")
        print("  📈 11_scatter_relationships.png - Scatter plots of relationships")
        print("  📈 12_time_series_trends.png - Performance trends across game")
        print(f"\n{'='*80}\n")


# Example usage
if __name__ == "__main__":
    # Load all result CSVs
    csv_pattern = "*_results.csv"  # This will match all files ending in _results.csv

    # Or specify explicitly:
    # csv_files = ["sft_model_results.csv", "grpo_model_results.csv", "baseline_results.csv"]

    comparison = ChessModelComparison(csv_pattern)
    comparison.generate_full_report(output_dir='chess_analysis_results')

Loaded 1 models: sft_model

GENERATING COMPREHENSIVE COMPARATIVE ANALYSIS


COMPREHENSIVE MODEL COMPARISON
    Model  Positions  Legal (%)  Best (%)  Top-3 (%)  Top-5 (%)  Format (%)  Avg CP Loss  Med CP Loss  Std CP Loss  Excellent (%)  Blunder (%)  Mistake (%)  Inaccuracy (%)  Draw→Loss (%)  Avg Time (s)  Avg Tokens
sft_model          3      33.33     16.67      16.67      16.67        0.00        20.50        20.50        23.33          50.00         0.00         0.00            0.00           0.00          6.58       64.33


Generating visualizations...
Saved: chess_analysis_results/01_core_metrics.png
Saved: chess_analysis_results/02_move_quality.png


/tmp/ipython-input-2264365787.py:269: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = axes[1, 1].boxplot(data_for_box, labels=labels_for_box, patch_artist=True)


Saved: chess_analysis_results/03_centipawn_analysis.png


/tmp/ipython-input-2264365787.py:353: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df[df['is_legal']].groupby('move_range')['is_best'].mean() * 100


Saved: chess_analysis_results/04_phase_analysis.png


...

Saved: chess_analysis_results/06_tactical_positional.png
Saved: chess_analysis_results/07_efficiency.png
Saved: chess_analysis_results/08_radar_comparison.png
Saved: chess_analysis_results/09_agreement_matrix.png
Saved: chess_analysis_results/10_elo_estimation.png


/tmp/ipython-input-2264365787.py:851: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axes[1, 0].legend()


Saved: chess_analysis_results/11_scatter_relationships.png


/tmp/ipython-input-2264365787.py:921: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axes[0, 1].legend()


Saved: chess_analysis_results/12_time_series_trends.png

✓ ANALYSIS COMPLETE!

All results saved to 'chess_analysis_results/' directory

Generated files:
  📊 summary_comparison.csv - Detailed metrics table
  📈 01_core_metrics.png - Core performance metrics
  📈 02_move_quality.png - Move quality breakdown
  📈 03_centipawn_analysis.png - Centipawn loss analysis
  📈 04_phase_analysis.png - Game phase performance
  📈 05_position_type.png - Position type analysis
  📈 06_tactical_positional.png - Tactical vs positional
  📈 07_efficiency.png - Speed and efficiency metrics
  📈 08_radar_comparison.png - Multi-dimensional radar chart
  📈 09_agreement_matrix.png - Model agreement heatmap
  📈 10_elo_estimation.png - Estimated playing strength
  📈 11_scatter_relationships.png - Scatter plots of relationships
  📈 12_time_series_trends.png - Performance trends across game


